In [1]:
!pip install shap

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ---------------------------------------- 0.0/456.1 kB ? eta -:--:--
      --------------------------------------- 10.2/456.1 kB ? eta -:--:--
      --------------------------------------- 10.2/456.1 kB ? eta -:--:--
      --------------------------------------- 10.2/456.1 kB ? eta -:--:--
     --- --------------------------------- 41.0/456.1 kB 178.6 kB/s eta 0:00:03
     --- --------------------------------- 41.0/456.1 kB 178.6 kB/s eta 0:00:03
     --- --------------------------------- 41.0/456.1 kB 178.6 kB/s eta 0:00:03
     --- --------------------------------- 41.0/456.1 kB 178.6 kB/s eta 0:00:03
     ----------- ------------------------ 143.4/456.1 kB 387.0 kB/s eta 0:00:01
     ----------- ------------------------ 143.4/456.1 kB 387.0 kB/s eta 0:00:01
     ----------- ------------------------ 143.4/456.1 kB 387.0 kB/s eta 0:00:01
     -------------------------- --------- 337.9/456.1 kB 635.0 kB/s eta 0:00:01
     

In [9]:
import torch
from transformers import ViTForImageClassification, ViTFeatureExtractor
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "Anwarkh1/Skin_Cancer-Image_Classification"
model = ViTForImageClassification.from_pretrained(model_name).to(device)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

dataset = load_dataset("marmal88/skin_cancer", split="test")

label_map = model.config.label2id

def transform(example):
    image = example["image"]
    image = feature_extractor(image, return_tensors="pt", size=(224, 224))["pixel_values"].squeeze(0)
    label_str = example["dx"]
    label_int = label_map.get(label_str, -1)
    if label_int == -1:
        raise ValueError(f"Unknown label {label_str}")
    return {"pixel_values": image, "dx": torch.tensor(label_int, dtype=torch.long)}

dataset = dataset.map(transform)
dataset.set_format(type="torch", columns=["pixel_values", "dx"])

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in dataloader:
        inputs = batch["pixel_values"].to(device)
        labels = batch["dx"].to(device)
        outputs = model(inputs)
        preds = torch.argmax(F.softmax(outputs.logits, dim=1), dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Using device: {device}")
print(f"CUDA Available: {torch.cuda.is_available()}")

accuracy = correct / total
print(f"Evaluation Accuracy: {accuracy:.4f}")

Using device: cuda
CUDA Available: True
Evaluation Accuracy: 0.8319


In [ ]:
print(dataset.column_names)


['image', 'image_id', 'lesion_id', 'dx', 'dx_type', 'age', 'sex', 'localization', 'pixel_values']


In [6]:
print(model.config.id2label)  

{0: 'benign_keratosis-like_lesions', 1: 'basal_cell_carcinoma', 2: 'actinic_keratoses', 3: 'vascular_lesions', 4: 'melanocytic_Nevi', 5: 'melanoma', 6: 'dermatofibroma'}


In [ ]:
import shap

class ModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x).logits

wrapped_model = ModelWrapper(model)

background_data = torch.stack([dataset[i]["pixel_values"] for i in range(10)]).to(device)

test_image = dataset[15]["pixel_values"].unsqueeze(0).to(device)

explainer = shap.GradientExplainer(wrapped_model, background_data)

shap_values = explainer.shap_values(test_image)

shap.image_plot(shap_values, test_image.cpu().numpy())
